In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from preprocessing import *
from sklearn.model_selection import KFold
from glob import glob

In [2]:
train_file_ds = tf.data.Dataset.list_files(['formatted_data/*/*.wav'])
batch_size = 5

In [3]:
train_ds = train_file_ds.map(get_audio_and_label).batch(batch_size)

for x, sampling_rate, label in train_ds.take(1):
    print(np.max(x), np.min(x))
    print(f'Data shape: {x.shape} - Sampling rate: {sampling_rate.numpy()} - Label: {label.numpy()}') # se non .batch c'è .decode()

Tensor("Shape:0", shape=(None,), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>
0.5000218 0.49999365
Data shape: (5, 192000, 2) - Sampling rate: [48000 48000 48000 48000 48000] - Label: [b'63' b'47' b'61' b'86' b'40']


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(batch_size, 192000, 2)),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
    ])

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.optimizers.Adam(learning_rate=1e-5)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

In [6]:
kf = KFold(n_splits=5, random_state=7, shuffle=True)
X = glob("complete_data/*.wav")
for i, (train_index, test_index) in enumerate(kf.split(X)):

    train_files = [X[j] for j in train_index]
    test_files = [X[j] for j in test_index]

    train_file_ds = tf.data.Dataset.list_files(train_files)
    print(train_file_ds)
    train_ds = train_file_ds.map(preprocess_with_resized_mel).batch(batch_size)
    for sampling_rate, example_label in train_ds.take(1):
        pass
    test_file_ds = tf.data.Dataset.list_files(test_files)
    test_ds = test_file_ds.map(preprocess_with_resized_mel).batch(batch_size)

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    history = model.fit(train_ds, epochs=5, validation_data=test_ds)
    # computing statistics
    test_loss, test_accuracy = model.evaluate(test_ds)
    # training_loss = history.history['loss'][-1]
    # training_accuracy = history.history['sparse_categorical_accuracy'][-1]
    # val_loss = history.history['val_loss'][-1]
    # val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]


<ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
Tensor("Shape:0", shape=(None,), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("Shape_1:0", shape=(None,), dtype=int32)
Tensor("Shape_2:0", shape=(None,), dtype=int32)


ValueError: in user code:

    File "/home/webbelle/quaroniSMS/preprocessing.py", line 112, in preprocess_with_resized_mel  *
        mfccs = tf.image.resize(mfccs, [32, 32])

    ValueError: 'images' contains no shape.
